<a href="https://colab.research.google.com/github/alex-wwwww/tacc-pukyu-yachay/blob/master/ProyectoCorpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, confusion_matrix

[nltk_data] Downloading package punkt to /home/nhrot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/nhrot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nhrot/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/nhrot/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [36]:
def transformar_numero_cadena(cadena):
    numeros = {
        '0': 'cero', '1': 'uno', '2': 'dos', '3': 'tres', '4': 'cuatro', '5': 'cinco', '6': 'seis', '7': 'siete', '8': 'ocho', '9': 'nueve',
        '10': 'diez', '11': 'once', '12': 'doce', '13': 'trece', '14': 'catorce', '15': 'quince', '16': 'dieciséis', '17': 'diecisiete', '18': 'dieciocho', '19': 'diecinueve',
        '20': 'veinte', '21': 'veintiuno', '22': 'veintidós', '23': 'veintitrés', '24': 'veinticuatro', '25': 'veinticinco', '26': 'veintiséis', '27': 'veintisiete', '28': 'veintiocho', '29': 'veintinueve',
        '30': 'treinta', '31': 'treinta y uno', '32': 'treinta y dos', '33': 'treinta y tres', '34': 'treinta y cuatro', '35': 'treinta y cinco', '36': 'treinta y seis', '37': 'treinta y siete', '38': 'treinta y ocho', '39': 'treinta y nueve',
        '40': 'cuarenta', '45': 'cuarenta y cinco', '50': 'cincuenta', '55': 'cincuenta y cinco', '60': 'sesenta', '65': 'sesenta y cinco', '70': 'setenta', '75': 'setenta y cinco', '80': 'ochenta', '85': 'ochenta y cinco', '90': 'noventa',
        '100': 'cien', '150': 'ciento cincuenta', '200': 'doscientos', '250': 'doscientos cincuenta', '300': 'trescientos', '350': 'trescientos cincuenta', '400': 'cuatrocientos', '450': 'cuatrocientos cincuenta', '500': 'quinientos', '550': 'quinientos cincuenta', '600': 'seiscientos', '650': 'seiscientos cincuenta', '700': 'setecientos', '750': 'setecientos cincuenta', '800': 'ochocientos', '850': 'ochocientos cincuenta', '900': 'novecientos',
        '1.': 'primero', '2.': 'segundo', '3.': 'tercero', '4.': 'cuarto', '5.': 'quinto', '6.': 'sexto', '7.': 'séptimo', '8.': 'octavo', '9.': 'noveno',
        'º.-': 'primer lugar', '2º.-': 'en segundo lugar', '3º.-': 'en tercer lugar', '4º.-': 'en cuarto lugar', '5º.-': 'en quinto lugar', '6º.-': 'en sexto lugar', '7º.-': 'en séptimo lugar', '8º.-': 'en octavo lugar', '9º.-': 'en noveno lugar',
        '/2': 'medio', '/4': 'cuarto', '/8': 'octavo', '/3': 'tercio'
        }
    
    cadena = cadena.lower()
    cadena = ' '.join(cadena.split())
    
    words = []
    for word in cadena.split():
        temp = word.split('/')
        if len(temp) > 1:
            if temp[0] in numeros.keys() and temp[1] in numeros.keys():
                words.append(numeros[temp[0]] + ' ' + numeros['/' + temp[1]])
        else:
            words.append(word)
    
    words = [ x if x not in numeros.keys() else numeros[x] for x in words ]
    cadena = ' '.join(words)
    
    words = []
    for word in cadena.split():
        temp = word.split('1')
        if len(temp) > 1:
            if temp[0] in numeros.keys() and temp[1] == '°.-':
                words.append(numeros[temp[0]] + ' ' + numeros[temp[1]])
        else:
            words.append(word)
    
    words = [ x if x not in numeros.keys() else numeros[x] for x in words ]
    cadena = ' '.join(words)
    return cadena  

def f_preprocesamiento(cadena):
  # Pasar a minusculas
  cadena = transformar_numero_cadena(cadena)
  # Eliminar espacios sobrantes
  cadena = ' '.join(cadena.split())
  # Tokenizar (No se solicita, pero se facilita procesamiento posterior)
  words = []
  words = cadena.split()
  cadena = ' '.join(words)
  words = []
  for sentence in sent_tokenize(cadena):
    words.append(word_tokenize(sentence))
  words = [x for l in words for x in l]
  # Procesar contracciones
  contract_dict = {'ud': 'unidad', 'kg': 'kilogramo', 'gr': 'gramo', 'ml': 'mililitro', 'dl': 'decilitro',
                    'cm': 'centímetro', 'mm': 'milímetro', 'm': 'metro', 'km': 'kilómetro', 'l': 'litro',
                    'seg': 'segundo', 'min': 'minuto', 'hr': 'hora', 'dia': 'día', 'mes': 'mes', 'año': 'año',
                   }
  words = [contract_dict[y] if y in contract_dict.keys() else y for y in words]
  # Eliminar grafías
  tokenizer = RegexpTokenizer(r"\w+")
  words = tokenizer.tokenize(' '.join(words))
  
  # Eliminar stopwords
  en_stopwords = stopwords.words('spanish')
  words =  [y for y in words if y not in en_stopwords or y == 'uno']
  
  cadena = ' '.join(words)
  cadena = transformar_numero_cadena(cadena)
  # La función debe devolver un string.
  return cadena

In [24]:
def procesar_texto(text_file):
    lineas = []
    with open(text_file, 'r') as file:
        for line in file:
            lineas.append(line)
    
    cadena = ""
    for linea in lineas:
        # add end of line
        cadena += f_preprocesamiento(linea) + '\n'
                
    return cadena

In [38]:
project_root = os.getcwd()
recetas_dir = os.path.join(project_root, 'recetas/recetas')
recetas_dir2 = os.path.join(project_root, 'recetas2')

recetas_files = os.listdir(recetas_dir)
recetas_files.sort()
recetas_files2 = os.listdir(recetas_dir2)
recetas_files2.sort()

print(procesar_texto(os.path.join(recetas_dir, recetas_files[0])))

receta albóndigas tomate
ingredientes
doscientos cincuenta gramo carne ternera picada
doscientos cincuenta gramo carne cerdo picada
cuatro rebanada pan molde
uno unidad huevo
uno chorro leche
uno chorro vino blanco
uno diente ajo
uno cucharada perejil picado
uno pizca pimienta blanca molida
uno cucharada sal
cien gramo harina
uno litro salsa tomate
instrucciones
primero pon rebanadas pan remojo buen chorro leche diez minutos
segundo mezcla carnes sazona sal pimienta aromatiza vino diente ajo picado brunoisecorte hortalizas verduras dados finos regulares
tercero añade pan remojado leche perejil picado huevo crudo amasa minutos
cuarto forma porciones menos veinticinco gramo dales forma bola puedes utilizar harina evitar peguen
quinto vez albóndigas formadas enharínalas fríelas abundante aceite caliente uno dos minutos doradas
sexto colócalas olla cazuela añade salsa tomate cocínalas fuego lento cuarenta minutos aproximadamente si necesario puedes añadir vaso agua ó caldo evitar salsa que

In [40]:
# Exportando recetas procesadas
recetas_procesadas_dir = os.path.join(project_root, 'recetas_procesadas')
os.makedirs(recetas_procesadas_dir, exist_ok=True)

# Limpiando directorio
for file in os.listdir(recetas_procesadas_dir):
    os.remove(os.path.join(recetas_procesadas_dir, file))

# Agregar recetas procesadas
for file in recetas_files:
    with open(os.path.join(recetas_procesadas_dir, file), 'w') as file_procesado:
        # print(file)
        file_procesado.write(procesar_texto(os.path.join(recetas_dir, file)))

for file in recetas_files2:
    with open(os.path.join(recetas_procesadas_dir, file), 'w') as file_procesado:
        # print(file)
        file_procesado.write(procesar_texto(os.path.join(recetas_dir2, file)))
